# **Practice #n + 7**  
##### 08.11.24


### **Тема**: Патерны. Комбинация



```mermaid
classDiagram
    class FinanceManager {
        +addTransaction(Transaction)
        +getTransactions() List~Transaction~
        +getBalance() double
    }

    class TransactionFactory {
        +createTransaction(amount, date, category) Transaction
    }

    class Transaction {
        -amount: double
        -date: Date
        -category: String
        +getAmount() double
        +getDate() Date
        +getCategory() String
    }

    class Category {
        -name: String
        +getName() String
    }

    class ReportGenerator {
        +generateMonthlyReport(List~Transaction~) Report
        +generateYearlyReport(List~Transaction~) Report
    }

    class Report {
        -transactions: List~Transaction~
        +getTransactions() List~Transaction~
    }

    FinanceManager "1" -- "1" TransactionFactory
    FinanceManager "1" -- "*" Transaction
    Transaction "1" -- "1" Category
    FinanceManager "1" -- "1" ReportGenerator
    ReportGenerator "1" -- "1" Report
```

In [21]:
from __future__ import annotations
from datetime import datetime


class Category:
    _categories = {}

    def __new__(cls, name):
        if name in Category._categories:
            return cls._categories[name]

        cat = super(Category, cls).__new__(cls)
        cls._categories[name] = cat
        return cat

    def __init__(self, name: str):
        if name in Category._categories:
            self = Category._categories[name]

        self.__name = name
        Category._categories[name] = self

    @property
    def name(self) -> str:
        return self.__name


class Transaction:
    def __init__(
        self, amount: float,
        date: datetime = datetime.now(),
        category: Category = Category("default")
    ):
        self.__amount = amount
        self.__date = date
        self.__category = category

    @property
    def amount(self) -> float:
        return self.__amount

    @property
    def date(self) -> datetime:
        return self.__date

    @property
    def category(self) -> Category:
        return self.__category


class Report:
    def __init__(self, transactions: list[Transaction]) -> None:
        self.__transactions = transactions

    @property
    def transactions(self) -> list[Transaction]:
        return self.__transactions

    def __str__(self):
        trans = []
        for tr in self.__transactions:
            s = f"{tr.amount} {tr.date} {tr.category.name}"
            trans.append(s)
        return "\n".join(trans)


class ReportGenerator():
    def generateReport(self, transactions: list[Transaction], specification: Specification) -> Report:
        res = []
        for tran in transactions:
            if specification.isSatisfied(tran):
                res.append(tran)
        return Report(res)


class Specification:
    def isSatisfied(self):
        pass


class MonthlySP(Specification):
    def __init__(self, month):
        self.specification = month

    def isSatisfied(self, transaction):
        return transaction.date.month == self.specification


class YearlySP(Specification):
    def __init__(self, year):
        self.specification = year

    def isSatisfied(self, transaction):
        return transaction.date.year == self.specification


class AndSP(Specification):
    def __init__(self, *args):
        self.specifications = args

    def isSatisfied(self, transaction):
        return all(spec.isSatisfied(transaction) for spec in self.specifications)


class TransactionFabric:
    pass


class FinanceManager:
    __transactionFabric = TransactionFabric()
    __reportGenerator = ReportGenerator()
    __transactions = []
    __balance = 0

    def __init__(self, transaction):
        self.transaction = transaction

    def addTransaction(self, transaction):
        self.transaction.append(transaction)

    def getTransaction(self):
        return self.transaction

    @property
    def getBalance(self):
        return sum(transaction.amount for transaction in self.transaction)

In [22]:
tr = Transaction(10)
tr2 = Transaction(10, category=Category("salary"))
tr3 = Transaction(10, category=Category("salary"))
rep = ReportGenerator().generateReport([tr, tr2, tr3], MonthlySP(11))
rep2 = ReportGenerator().generateReport([tr, tr2, tr3], MonthlySP(12))
rep3 = ReportGenerator().generateReport(
    [tr, tr2, tr3], AndSP(MonthlySP(11), YearlySP(2024)))
rep.transactions, rep2.transactions
rep3.transactions

In [23]:
from copy import deepcopy


class FinanceManagerState:
    def __init__(self, balance, transactions):
        self.balance = balance
        self.transactions = deepcopy(transactions)


class Memento:
    __sate = None

    @classmethod
    def saveState(cls, manager: FinanceManager):
        cls.__sate = FinanceManagerState(manager.balance, manager.transactions)

    @classmethod
    def loadState(cls, manager: FinanceManager):
        return cls.__sate.transactions, cls.__sate.balance

In [24]:
from __future__ import annotations
from datetime import datetime


class FinanceManager:
    __reportGenerator = ReportGenerator()
    transactions = []
    __balance = 0

    @property
    def tranasactions(self):
        return self.__transactions

    @property
    def balance(self):
        return FinanceManager.__balance

    def addTransaction(self, amount, date, category):
        Memento.saveState(self)
        try:
            tran = Transaction(amount, date, category)
            self.__balance += amount
            self.transactions.append(tran)
        except Exception as e:  # noqa: E722
            self.tranasactions, self.balance = Memento.loadState()
            print(e)

            # raise Exception("Failed creating of new Transaction")

    def report(self):
        return self.__reportGenerator.generateReport(self.transactions, MonthlySP(11))


manager = FinanceManager()

tr = Transaction(10)
tr2 = Transaction(10, category=Category("salary"))
tr3 = Transaction(10, category=Category("salary"))
manager.addTransaction(10, datetime.now(), Category("salary"))
print(manager.report())
manager.addTransaction("10", datetime.now(), Category("boombastic"))
print(manager.report())

10 2024-11-22 13:44:13.708590 salary


TypeError: Category.__new__() missing 1 required positional argument: 'name'